In [1]:
import cmipld 



In [3]:
# import cmipld
# from cmipld.utils.ldparse import name_extract

# me = __file__.split('/')[-1].replace('.py','')

# def run(io, whoami, path, name, **kwargs):
#     data = cmipld.get(f"{io}/{me}/graph.jsonld", depth=0)["@graph"]
#     summary = name_extract(data)
#     return f"{path}/{name}_{me}.json", me, summary

In [4]:
from cmipld import jsonld 
import json

In [9]:
exp = cmipld.get('https://wcrp-cmip.github.io/CMIP7-CVs/experiment/amip.json',depth=2)
exp

{'@context': 'https://wcrp-cmip.github.io/CMIP7-CVs/experiment/amip.json',
 '@id': 'amip',
 '@type': ['wcrp:experiment', 'esgvoc:Experiment', 'cmip7'],
 'activity': [{'@id': 'damip',
   '@type': ['wcrp:activity', 'esgvoc:Activity', 'universal'],
   'description': 'DAMIP focuses on detection and attribution of climate change, covering experiments like hist-aer and hist-nat.',
   'ui_label': 'Detection and Attribution Model Intercomparison Project',
   'url': 'https://gmd.copernicus.org/articles/13/7947/2020/gmd-13-7947-2020.pdf',
   'validation_key': 'DAMIP'},
  {'@id': 'cfmip',
   '@type': ['wcrp:activity', 'esgvoc:Activity', 'universal'],
   'description': 'CFMIP experiments focus on cloud feedbacks and their role in climate, including abrupt-0p5CO2 and amip-piForcing simulations.',
   'ui_label': 'Cloud Feedback Model Intercomparison Project ',
   'url': 'https://gmd.copernicus.org/articles/10/359/2017/gmd-10-359-2017.pdf',
   'validation_key': 'CFMIP'}],
 'alias': [],
 'description'

In [66]:
from cmipld.utils.ldparse import cvjson_validation_key

def ld2cmip6(validation_key, ui_label, description, activity, 
                 parent_experiment, tier, model_realms, 
                 start="", end="", min_years=0, parent_activity=None, **kwargs):
    """Convert form data to experiment JSON format."""

    model_realms = model_realms if isinstance(model_realms, list) else [model_realms]

    required = [m for m in model_realms if m.get('is_required')]
    optional = [m for m in model_realms if not m.get('is_required')]
    
    
    print(required)

    return {
        validation_key: {
            "activity": cvjson_validation_key(activity),
            "additional_allowed_model_components": cvjson_validation_key(optional),
            "description": description,
            "end": end,
            "experiment": ui_label,
            "experiment_id": validation_key,
            "min_number_yrs_per_sim": min_years,
            "parent_activity_id": cvjson_validation_key(parent_activity) if parent_activity else [],
            "parent_experiment_id": cvjson_validation_key(parent_experiment),
            "required_model_components":cvjson_validation_key(required),
            "start": start,
            "tier": tier
        }
    }

ImportError: cannot import name 'cvjson_validation_key' from 'cmipld.utils.ldparse' (/Users/daniel.ellis/WIPwork/CMIP-LD/cmipld/utils/ldparse.py)

In [64]:
data = cmipld.get('cmip7:experiment/graph.jsonld', depth=2)

import tqdm
complete = {}

for i in tqdm.tqdm(data['@graph']):
    complete.update(ld2cmip6(**i))
    

    
complete = dict(sorted(complete.items()))
complete

100%|██████████| 120/120 [00:00<00:00, 172427.71it/s]

[{'@id': 'aogcm', '@type': ['wcrp:source_type', 'esgvoc:SourceType', 'universal'], 'description': 'coupled atmosphere-ocean global climate model, additionally including explicit representation of at least the land and sea ice', 'is_required': True, 'ui_label': '', 'validation_key': 'AOGCM'}, {'@id': 'bgc', '@type': ['wcrp:source_type', 'esgvoc:SourceType', 'universal'], 'description': 'biogeochemistry model component that at the very least accounts for carbon reservoirs and fluxes in the atmosphere, terrestrial biosphere, and ocean', 'is_required': True, 'ui_label': '', 'validation_key': 'BGC'}]
[{'@id': 'aogcm', '@type': ['wcrp:source_type', 'esgvoc:SourceType', 'universal'], 'description': 'coupled atmosphere-ocean global climate model, additionally including explicit representation of at least the land and sea ice', 'is_required': True, 'ui_label': '', 'validation_key': 'AOGCM'}, {'@id': 'bgc', '@type': ['wcrp:source_type', 'esgvoc:SourceType', 'universal'], 'description': 'biogeoch

{'1pctCO2': {'activity': ['DAMIP', 'PMIP'],
  'additional_allowed_model_components': ['AER', 'CHEM', 'BGC'],
  'description': 'DECK: 1 percent per year CO2 increase \n Increase CO2 concentration from pre-industrial level at 1% per year until quadrupling',
  'end': -999,
  'experiment': 'CO2 prescribed to increase at 1% per year',
  'experiment_id': '1pctCO2',
  'min_number_yrs_per_sim': 0,
  'parent_activity_id': [],
  'parent_experiment_id': ['picontrol'],
  'required_model_components': ['AOGCM'],
  'start': -999,
  'tier': 0},
 '1pctCO2-bgc': {'activity': ['fast-track', 'DAMIP'],
  'additional_allowed_model_components': ['AER', 'CHEM', 'ISM'],
  'description': '1pctCO2-bgc is retained from CMIP6.  It is a simulation which branches from piControl with a 1% per year increase in CO2 concentration from pre-industrial levels, with the CO2 “biogeochemically-coupled”. This means that only the model’s carbon cycle components (both land and ocean) respond to the increase in CO2, whereas the m

In [72]:
from cmipld.utils.ldparse import name_entry

activities = name_entry(cmipld.get('cmip7:project/activity.json', depth=2)['activity'])

activities

OrderedDict([('AerChemMIP',
              'AerChemMIP experiments focus on the role of atmospheric chemistry and aerosols in climate, including piClim and hist-piSLCF simulations.'),
             ('C4MIP',
              'C4MIP experiments focus on carbon cycle feedbacks and interactions, including 1pctCO2-bgc and esm-flat10-cdr experiments.'),
             ('CFMIP',
              'CFMIP experiments focus on cloud feedbacks and their role in climate, including abrupt-0p5CO2 and amip-piForcing simulations.'),
             ('CMIP',
              'CMIP DECK: 1pctCO2, abrupt-4xCO2, amip, esm-piControl, esm-historical, historical, and piControl experiments'),
             ('DAMIP',
              'DAMIP focuses on detection and attribution of climate change, covering experiments like hist-aer and hist-nat.'),
             ('DCPP',
              'DCPP focuses on decadal climate predictions and initialized experiments like prediction for 2025-2036.'),
             ('GeoMIP',
              'GeoM

In [76]:
from cmipld.utils.ldparse import name_entry

mip_era = cmipld.get('cmip7:project/mipera.json', depth=2)

mip_era

JsonLdError: ('Cyclical @context URLs detected.',)
Type: jsonld.ContextUrlError
Code: context overflow
Details: {'url': 'https://wcrp-cmip.github.io/CMIP7-CVs/project/mipera.json'}